In [79]:
import torch
from transformers import AutoTokenizer, BertTokenizer, ElectraConfig, ElectraTokenizerFast
from transformers import AutoModel, BertModel, ElectraForMaskedLM, AutoConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# tokenizer = AutoTokenizer.from_pretrained('SpanBERT/spanbert-large-cased')
# AutoConfig.from_pretrained('/Users/amritalok/Downloads/spanbert_hf_base')
# tokenizer = AutoTokenizer.from_pretrained('/Users/amritalok/Downloads/spanbert_hf_base')
# tokenizer = ElectraTokenizerFast.from_pretrained('google/electra-small-generator')
# model = AutoModel.from_pretrained('SpanBERT/spanbert-base-cased') # the path to .bin and config.json

# gen_config = ElectraConfig.from_pretrained('google/electra-large-generator')
# disc_config = ElectraConfig.from_pretrained('google/electra-large-discriminator')
# gen_config.hidden_size = disc_config.hidden_size // 4
# gen_config.num_attention_heads = disc_config.num_attention_heads // 4
# gen_config.intermediate_size = disc_config.intermediate_size // 4


In [87]:
# sentence = "[CLS] Panda conservation, on its own, has been a success. [CLS] In the 1980s, pandas had very little protection in the wild and were thought to number around just 1,100 individuals. [CLS]"
sentence = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
# sentence = "The capital of France is Paris"

In [88]:
tokenized_text = tokenizer.tokenize(sentence)

In [89]:
masked_indices = [11]
# masked_indices = [22, 23]
for masked_index in masked_indices:
    tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

['[CLS]', 'Who', 'was', 'Jim', 'He', '##nson', '?', '[SEP]', 'Jim', 'He', '##nson', '[MASK]', 'a', 'puppet', '##eer', '[SEP]']


In [90]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [91]:
# Span_BERT
# segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# Span BERT
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
# ELECTRA and BERT
# segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
# segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# segments_ids = [0, 0, 0, 0, 0, 0]

print(len(indexed_tokens), len(segments_ids))

16 16


In [92]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [93]:
from transformers import AutoModelForMaskedLM #BertForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('/Users/amritalok/Downloads/spanbert_hf_base')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator')
model.eval()

# If you have a GPU, put everything on cuda
# tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
# model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
for masked_index in masked_indices:
    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)

Some weights of BertForMaskedLM were not initialized from the model checkpoint at /Users/amritalok/Downloads/spanbert_hf_base and are newly initialized: ['embeddings.word_embeddings.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.LayerNorm.weight', 'embeddings.LayerNorm.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.Layer

['deny']


In [103]:
# from collections import OrderedDict
# import torch

# input_path = '/Users/amritalok/Downloads/spanbert_large_with_head/model.pt'
# output_path = '/Users/amritalok/Downloads/SpanBertLarge/pytorch_model.bin'

# def convert(input_path, output_path):
#     state_dict = OrderedDict()
#     for k, v in torch.load(input_path, map_location=torch.device('cpu')).items():
#         if k.startswith('decoder.'):
#             state_dict[k[8:]] = v
#     torch.save(state_dict, output_path)


# convert(input_path, output_path)

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
model = BertForMaskedLM.from_pretrained('/Users/amritalok/Downloads/SpanBertLarge/')

seq = 'Panda conservation, on its own, has been a {m}. In the 1980s, pandas had very little {m} in the wild and were {m} to number around just 1,100 individuals.'.format(m=tokenizer.mask_token)

# seq = 'Who was Jim Henson ? Jim {m} {m} {m} a puppeteer'.format(m=tokenizer.mask_token)

# seq = "Super Bowl 50 was {m} {m} {m} {m} to determine the champion".format(m=tokenizer.mask_token)
inputs = tokenizer(seq, return_tensors="pt")
outputs = model(**inputs).logits

print(tokenizer.decode([y if x == tokenizer.mask_token_id else x for x, y
                        in zip(inputs.input_ids[0], outputs.argmax(dim=2)[0])]))

Some weights of the model checkpoint at /Users/amritalok/Downloads/SpanBertLarge/ were not used when initializing BertForMaskedLM: ['cls.pair_target_predictions.bias', 'cls.pair_target_predictions.position_embeddings.weight', 'cls.pair_target_predictions.mlp_layer_norm.linear1.weight', 'cls.pair_target_predictions.mlp_layer_norm.linear1.bias', 'cls.pair_target_predictions.mlp_layer_norm.linear2.weight', 'cls.pair_target_predictions.mlp_layer_norm.linear2.bias', 'cls.pair_target_predictions.decoder.weight', 'cls.pair_target_predictions.mlp_layer_norm.layer_norm1.weight', 'cls.pair_target_predictions.mlp_layer_norm.layer_norm1.bias', 'cls.pair_target_predictions.mlp_layer_norm.layer_norm2.weight', 'cls.pair_target_predictions.mlp_layer_norm.layer_norm2.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model)

[CLS] Panda conservation, on its own, has been a challenge. In the 1980s, pandas had very little success in the wild and were estimated to number around just 1, 100 individuals. [SEP]
